In [1]:
# importing Library
import pandas as pd
import numpy as np
import pdfplumber

import re
import os
import string
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vikas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# DATA GATHERING

In [2]:
# Grathering all file from directory

all_files = []

for path, subdirs, files in os.walk(r'C:/Users/vikas/Desktop/articles'):
    for name in files:
        all_files.append(os.path.join(path, name))

In [3]:
# Loop through the all file and extracting text

all_text=[]
for f in tqdm(all_files):
    with pdfplumber.open(f) as pdf:
        pdf_text = []
        pdf_text.append(f)
        for page in pdf.pages:
            text = page.extract_text(x_tolerance=0, y_tolerance=0)
            text = text.lower()
            text = re.sub('†','', text)
            text = re.sub('‡','', text)
            text = re.sub('∗','', text)
            text = re.sub('\[.*?\]', '', text)
            text = re.sub('https?://\S+|www\.\S+', '', text)
            text = re.sub('<.*?>+', '', text)
            text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
            text = re.sub('\n', '', text)
            text = re.sub('\w*\d\w*', '', text)
            pdf_text.append(text)
        all_text.append(pdf_text)


100%|██████████| 10/10 [00:40<00:00,  4.00s/it]


In [4]:
# preview

all_text[1][1:2]

['a retrospective analysis of the fake news challengestance detection task   andreas hanselowski  avinesh pvs  benjamin schiller  felix caspelherr    debanjan chaudhuri  christian m meyer  iryna gurevychresearch training group aiphescomputer science department technische universität darmstadtsmart data analytics university of bonn darmstadtdeabstractthe  fake news challenge stage   shared task addressed a stance classiﬁcationtask as a crucial ﬁrst step towards detecting fake news to date there is no indepth analysis paperto critically discuss ’s experimental setup reproduce the results and draw  nextgeneration stance classiﬁcation methods in this paper we provide such an  for the three topperforming systems we ﬁrst ﬁnd that ’s proposed  favors the majority class which can be easily classiﬁed and thus overestimates the  discriminative power of the methods therefore we propose a new  metric yieldingna changed system ranking next we compare the features and architectures used which leadsu

In [5]:
# Coverting List of text to dataframe
items = []
for item in all_text:
    items.append(item[1:])

df = pd.DataFrame(
    {'text': [items]}) 
df = df.explode('text').reset_index(drop=True)

# List of dates
publish_date = ['27-01-2020', '13-06-2018', '08-02-2019', '08-08-2019','03-11-2019', '06-01-2020', '18-03-2020', '07-04-2020','01-03-2020', '03-11-2019']

# List of title
title = ['Sentiment Classification Based on Part-of-Speech and Self-Attention Mechanism',
'A Retrospective Analysis of the Fake News Challenge Stance Detection Task',
'Detecting Incongruity Between News Headline and Body Text via a Deep Hierarchical Encoder',
'Your Stance is Exposed! Analysing Possible Factors for Stance Detection on Social Media',
'Part of Speech Tagging for Code Switched Data',
'Stance Detection Benchmark: How Robust Is Your Stance Detection?',
'A Multilingual Multi-Target Dataset for Stance Detection',
'A Few Topical Tweets are Enough for Effective Stance Detection',
'Generating Summaries Through Unigram and Bigram: Text Summarization',
'Incorporating Label Dependencies in Multilabel Stance Detection',]

# Mergering list to dataframe
df['publish_date'] = publish_date 
df['title'] = title

# sort by date
df['publish_date'] =pd.to_datetime(df.publish_date)
df.sort_values(by='publish_date',ascending=False, inplace=True, )



# rearrange the columns
df = df[['publish_date', 'title','text']].reset_index(drop=True)
df1 = df.copy()


# Trend Analysis
For the trend analysis we can calculate word count of document know the distribution and extract the word frequency and keyword over the year will show the usage of words in particular year and how trend is changeing in particular feild.  

In [6]:
# cleaing the text
df['clean_text'] = [" ".join(text) for text in df['text'].values]

def text_preprocessing(text):
    stop_words = set(stopwords.words("english"))
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    tokenized_text = tokenizer.tokenize(text)
    remove_stopwords = [w for w in tokenized_text if w not in stop_words]
    combined_text = ' '.join(remove_stopwords)
    return combined_text

df['clean_text'] = df['clean_text'].apply(lambda x : text_preprocessing(x))

# Fetch word count for each text
df['word_count'] = df['clean_text'].apply(lambda x: len(str(x).split(" ")))

In [7]:
df

,publish_date,title,text,clean_text,word_count
0,2020-07-04,A Few Topical Tweets are Enough for Effective ...,[a few topical tweets are enough for effective...,topical tweets enough effective stance detecti...,3205
1,2020-06-01,Stance Detection Benchmark: How Robust Is Your...,[stance detection benchmark how robust is your...,stance detection benchmark robust stance detec...,3663
2,2020-03-18,A Multilingual Multi-Target Dataset for Stance...,[stancexa multilingual multitarget dataset for...,stancexa multilingual multitarget dataset stan...,3278
3,2020-01-27,Sentiment Classification Based on Part-of-Spee...,[received january accepted january date of...,received january accepted january date publica...,3896
4,2020-01-03,Generating Summaries Through Unigram and Bigra...,[inte rna tiona l j ournal of information te c...,inte rna tiona l j ournal information te chnol...,4831
5,2019-08-08,Your Stance is Exposed! Analysing Possible Fac...,[your stance is exp ose d analysing possible f...,stance exp ose analysing possible factors fors...,7875
6,2019-08-02,Detecting Incongruity Between News Headline an...,[detecting incongruity between news headline a...,detecting incongruity news headline body textv...,4193
7,2019-03-11,Part of Speech Tagging for Code Switched Data,[part of speech tagging for code switched data...,part speech tagging code switched data fahad a...,3089
8,2019-03-11,Incorporating Label Dependencies in Multilabel...,[incorporating label dependencies in multilabe...,incorporating label dependencies multilabel st...,1701
9,2018-06-13,A Retrospective Analysis of the Fake News Chal...,[a retrospective analysis of the fake news cha...,retrospective analysis fake news challengestan...,4911


In [8]:
df['clean_text'][[0]]

0    topical tweets enough effective stance detecti...
Name: clean_text, dtype: object

In [9]:
# Building the vocabilary
stop_words = set(stopwords.words("english"))
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    stop_words=stop_words,
    ngram_range=(1, 2),
    max_features=10000)

X=word_vectorizer.fit_transform(df['clean_text'])

In [10]:
from scipy.sparse import coo_matrix
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(X)
# get feature names
feature_names=word_vectorizer.get_feature_names()
 
# fetch document for which keywords needs to be extracted
article=df['clean_text'][[6]]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(word_vectorizer.transform(article))


In [12]:
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,5)
keyword = []
for k in keywords:
    keyword.append(k)
keyword

['body text', 'incongruent', 'incongruence', 'ip', 'ahde']

In [13]:
keyword = [['expanded', 'darwish', 'retweeted', 'unsupervised classification','expanded test'], 
            ['resilience', 'spelling', 'biases', 'attacks','std datasets','related tasks','low resource'],
            ['answers', 'french', 'switzerland', 'swiss', 'crosstarget'], 
            ['focal', 'mechanism', 'focal loss', 'conf', 'sentimental'],
            ['summarization','phological','classif','summaries'], 
            ['body text', 'incongruent', 'incongruence', 'ip', 'ahde'],
            ['stancedete','predetection', 'tweets', 'performance','model'], 
            ['monolingual taggers', 'madamira', 'language pairs',  'pos taggers', 'condition','arabic'], 
            ['bbc', 'mtlxld', 'mftc', 'multilabel', 'multiclass'], 
            ['headline document', 'headline', 'gaza', 'bound', 'athene']]

In [14]:
# Mergering list to dataframe
df['keyword'] = keyword
df 

,publish_date,title,text,clean_text,word_count,keyword
0,2020-07-04,A Few Topical Tweets are Enough for Effective ...,[a few topical tweets are enough for effective...,topical tweets enough effective stance detecti...,3205,"[expanded, darwish, retweeted, unsupervised cl..."
1,2020-06-01,Stance Detection Benchmark: How Robust Is Your...,[stance detection benchmark how robust is your...,stance detection benchmark robust stance detec...,3663,"[resilience, spelling, biases, attacks, std da..."
2,2020-03-18,A Multilingual Multi-Target Dataset for Stance...,[stancexa multilingual multitarget dataset for...,stancexa multilingual multitarget dataset stan...,3278,"[answers, french, switzerland, swiss, crosstar..."
3,2020-01-27,Sentiment Classification Based on Part-of-Spee...,[received january accepted january date of...,received january accepted january date publica...,3896,"[focal, mechanism, focal loss, conf, sentimental]"
4,2020-01-03,Generating Summaries Through Unigram and Bigra...,[inte rna tiona l j ournal of information te c...,inte rna tiona l j ournal information te chnol...,4831,"[summarization, phological, classif, summaries]"
5,2019-08-08,Your Stance is Exposed! Analysing Possible Fac...,[your stance is exp ose d analysing possible f...,stance exp ose analysing possible factors fors...,7875,"[body text, incongruent, incongruence, ip, ahde]"
6,2019-08-02,Detecting Incongruity Between News Headline an...,[detecting incongruity between news headline a...,detecting incongruity news headline body textv...,4193,"[stancedete, predetection, tweets, performance..."
7,2019-03-11,Part of Speech Tagging for Code Switched Data,[part of speech tagging for code switched data...,part speech tagging code switched data fahad a...,3089,"[monolingual taggers, madamira, language pairs..."
8,2019-03-11,Incorporating Label Dependencies in Multilabel...,[incorporating label dependencies in multilabe...,incorporating label dependencies multilabel st...,1701,"[bbc, mtlxld, mftc, multilabel, multiclass]"
9,2018-06-13,A Retrospective Analysis of the Fake News Chal...,[a retrospective analysis of the fake news cha...,retrospective analysis fake news challengestan...,4911,"[headline document, headline, gaza, bound, ath..."


**The above short analysis show the word count distribution over time and extracted word freqency and keyword over the year to know the trend.**

# Text summarization

There is different type of text summarization methods and libraries used for summarization such as NLTK, Spacy and deep learing. For this project I am using Spacy to demonstrate the Text summarization.

### 1. Convert Paragraphs to Sentences
We first need to convert the whole paragraph into sentences. The most common way of converting paragraphs to sentences is to split the paragraph whenever a period is encountered.

### 2. Text Preprocessing
After converting paragraph to sentences, we need to remove all the special characters, stop words and numbers from all the sentences.

### 3. Tokenizing the Sentences
We need to tokenize all the sentences to get all the words that exist in the sentences

### 4. Find Weighted Frequency of Occurrence
Next we need to find the weighted frequency of occurrences of all the words. We can find the weighted frequency of each word by dividing its frequency by the frequency of the most occurring word.

### 5. Replace Words by Weighted Frequency in Original Sentences
The final step is to plug the weighted frequency in place of the corresponding words in original sentences and finding their sum. It is important to mention that weighted frequency for the words removed during preprocessing (stop words, punctuation, digits etc.) will be zero and therefore is not required to be added

### 6. Sort Sentences in Descending Order of Sum
The final step is to sort the sentences in inverse order of their sum. The sentences with highest frequencies summarize the text.

In [15]:
import spacy
import heapq
import warnings
import logging
from spacy import displacy

nlp = spacy.load('en_core_web_sm')
warnings.filterwarnings('ignore')
logging.getLogger("lda").setLevel(logging.WARNING)
stopwords = stopwords.words('english')

In [16]:
# Clean text before feeding it to spaCy
punctuations = '!"#$%&\'()*+,-/:;<=>?@[\\]^_`{|}~'
# Define function to cleanup text by removing personal pronouns, stopwords, and puncuation
def cleanup_text(docs, logging=False):
    texts = []
    doc = nlp(docs, disable=['parser', 'ner'])
    tokens = [tok.lemma_.lower().strip() for tok in doc if tok.lemma_ != '-PRON-']
    tokens = [tok for tok in tokens if tok not in stopwords and tok not in punctuations]
    tokens = ' '.join(tokens)
    texts.append(tokens)
    return pd.Series(texts)


df['clean_text_sp'] = df['clean_text'].apply(lambda x: cleanup_text(x, False))

In [17]:
# Function for text summarization

def generate_summary(text_without_removing_dot, cleaned_text):
    sample_text = text_without_removing_dot
    doc = nlp(sample_text)
    sentence_list=[]
    for idx, sentence in enumerate(doc.sents): # we are using spacy for sentence tokenization
        sentence_list.append(re.sub(r'[^\w\s]','',str(sentence)))

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}  
    for word in nltk.word_tokenize(cleaned_text):  
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1


    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():  
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)


    sentence_scores = {}  
    for sent in sentence_list:  
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 30:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]


    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)

    summary = ' '.join(summary_sentences)
    print("Original Text::::::::::::\n")
    print(text_without_removing_dot)
    print('\n\nSummarized text::::::::\n')
    print(summary)

In [100]:
generate_summary(df['clean_text'][0][:10000], df['clean_text_sp'][0])

Original Text::::::::::::

topical tweets enough effective stance detectionyounes samih kareem darwishqatar computing research institutehamad bin khalifa university doha qatar ysamihkdarwish detection entails ascertaining position target entity topic claim employs unsupervised classiﬁcation performing stance detection vocal twitter users rahave many tweets target yield high however methods perform poorly fail comapletely less vocal users may authored tweets target paper tackle stance users using two approaches ﬁrst ap proach improve userlevel stance detection representbiing tweets using contextualized embeddings captureslatent meanings words context show apfigure sample us midterm election related tweetsproach outperforms two strong baselines achieves macro fmeasure eight controversialcthat either express clear stance btopics second approach expand tweets given user using twitter timeline tweets unsupervised classiﬁcation user enoptimal results users rarely express opinionvtails cluste

In [101]:
generate_summary(df['clean_text'][1][:10000], df['clean_text_sp'][1])

Original Text::::::::::::

stance detection benchmark robust stance detectionbenjamin schiller johannes daxenberger iryna gurevychubiquitous knowledge processing lab ukptuda department computer science technische universitat darmstadt darmstadtdeabstracttopicclimate change real concerntweetgone days would getstance detection std aims detect autemperatures min max cape townsemstthor stance towards certain topic claimstancefavorand become key component like fake news detection claim task long tradition domain argument search however whilelitical ideological online debates easily detected humans machine et al walker et al somasundarannlearning models clearly falling short thisaand wiebe thomas et al recenttask given major differences datasetjsizes framing std eg number ofyears brought focus attention inputs introduce std benchby uprising debates around fake news mark learns ten std datasets ofstd important preprocessing step pomerleaulmultidataset learningvarious domains aand rao derczyns

In [103]:
generate_summary(df['clean_text'][2][:10000], df['clean_text_sp'][2])

Original Text::::::::::::

stancexa multilingual multitarget dataset stance detectionjannis vamvas rico computational linguistics university informatics university paper propose much larger datasetthat combines multilinguality multitude ofwe extract largescale stance detection datatopics targets stance comprises thanxset comments written candidates questions concerning swiss politics moreelections switzerland dataset answers given last decade german french italian text running political ofﬁce switzerlandlowing crosslingual evaluation stance questions available four languages enrdetection contains comments onatargetsmore political issues unglish swiss standard german french italmlike stance detection models speciﬁcian language comment depends target issues use dataset train region model issues make learnwe extracted data voting ad ing across targets possible prepend application platforminstance natural question represents thelcandidates respond questions mainly categortarget eg support 

** We successfully develop a model can summaries the text by running above fuction will give as summaries text.**

# Extracting Main Entities

In [113]:
from pprint import pprint
from collections import Counter
pprint([(X.text, X.label_) for X in doc.ents])

# Extracting named entity from an article
tagged_text = nlp(df['clean_text'][0])
extracted_entities = [(i.text, i.label_) for i in tagged_text.ents]

pprint(extracted_entities)

[('andrea hanselowski', 'PERSON'),
 ('benjamin schiller', 'PERSON'),
 ('debanjan', 'PERSON'),
 ('christian meyer iryna', 'ORG'),
 ('science department technische universität darmstadtsmart', 'ORG'),
 ('data analytics university', 'ORG'),
 ('bonn', 'GPE'),
 ('three', 'CARDINAL'),
 ('ﬁnd', 'ORG'),
 ('ai technology automatically detect fake news challenge', 'ORG'),
 ('nlp community', 'ORG'),
 ('four', 'CARDINAL'),
 ('essay stab gurevych', 'PERSON'),
 ('fourth', 'ORDINAL'),
 ('three', 'CARDINAL'),
 ('palestinians', 'NORP'),
 ('israel', 'GPE'),
 ('damsagreegaza city', 'GPE'),
 ('hundred', 'CARDINAL'),
 ('palestinians', 'NORP'),
 ('sunday', 'DATE'),
 ('morning', 'TIME'),
 ('israeli', 'NORP'),
 ('ﬂoode gaza valley', 'GPE'),
 ('recenta g', 'PERSON'),
 ('gaza ministry', 'ORG'),
 ('gaza', 'GPE'),
 ('three meter', 'QUANTITY'),
 ('hundred', 'CARDINAL'),
 ('israel', 'GPE'),
 ('gaza strip', 'GPE'),
 ('israel', 'GPE'),
 ('gaza strip', 'GPE'),
 ('israel', 'GPE'),
 ('palestinian', 'NORP'),
 ('thegaza v

In [114]:
# Unique Lables
labels = [i.label_ for i in tagged_text.ents]
Counter(labels)

Counter({'PERSON': 55,
         'GPE': 12,
         'CARDINAL': 23,
         'ORDINAL': 5,
         'NORP': 14,
         'ORG': 21,
         'DATE': 3,
         'PRODUCT': 1})

In [115]:
# Extracting the main entities
relevant_labels = ["NORP", "PERSON", "ORG", "GPE"]

for relevant_label in relevant_labels:
    print("Extracted for label: " + relevant_label)
    for entity, label in extracted_entities:
        if label == relevant_label:
            print("- " + entity)
            print("\n")

Extracted for label: NORP
- republican


- batayneh


- feb mar


- notgun


- republican


- examinedpolitbayesian


- egyptian


- stron


- republican


- islamophobic


- parisattacksthus


- north american


- evenamerican


- islamist


Extracted for label: PERSON
- institutehamad bin khalifa university


- et al magdy


- usepopescu wong


- et al


- zarrella marsh


- bert


- thomas pang lee moboth


- al cohen


- colleoni rozza arvidsson


- makazhanov raﬁei


- nakov


- mohammad et


- kavanaugh usand


- feb mar


- bert stance classiﬁcationgun


- feb mar omar


- mar


- omar mar controversial


- thegiven


- bcloser


- et al


- classiﬁer svmstefanov et al stefanov


- nakov esticlassiﬁer


- joulin et al sinceto


- svm svm


- et al bert


- theand umlfit


- howard ruder


- bert


- roberta liu et


- xlm lample


- bert perbasemultilingualwhen


- svm bertr


- ali


- jeff ﬂakestances


- unusablert somewe


- bert fasttext


- bert


- bert


- k magdy


- au

In [119]:
# Visualizing named entities
displacy_image = displacy.render(nlp(df['clean_text'][0][:5000]), jupyter = True, style = 'ent')